In [35]:
import os
import findspark
findspark.init(os.getenv('SPARK_HOME'))
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import MulticlassMetrics

In [2]:
spark = SparkSession.builder.appName("logreg_demo").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/17 11:10:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
data = spark.read.format('libsvm').load('sample_libsvm_data.txt')
data.show()

22/08/17 11:11:34 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [8]:
log_reg = LogisticRegression()

In [9]:
log_reg_model = log_reg.fit(data)

22/08/17 11:17:01 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/08/17 11:17:01 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [10]:
summary = log_reg_model.summary

In [12]:
summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [16]:
train, test = data.randomSplit([0.8, 0.2])

In [17]:
log_reg = LogisticRegression()

In [18]:
log_reg_model = log_reg.fit(train)

In [30]:
results = log_reg_model.evaluate(test)

In [31]:
results.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[23.0235876145530...|[0.99999999989977...|       0.0|
|  0.0|(692,[100,101,102...|[1.14052848281409...|[0.75777665587053...|       0.0|
|  0.0|(692,[122,123,148...|[28.7361314575761...|[0.99999999999966...|       0.0|
|  0.0|(692,[123,124,125...|[40.0002163517985...|           [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|[28.3967594240273...|[0.99999999999953...|       0.0|
|  0.0|(692,[124,125,126...|[61.0215706448290...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[27.7073959638815...|[0.99999999999907...|       0.0|
|  0.0|(692,[126,127,128...|[16.7509853801263...|[0.99999994689450...|       0.0|
|  0.0|(692,[128,129,130...|[23.1496903544705...|[0.99999999991164...|       0.0|
|  0.0|(692,[129

In [70]:
results.accuracy, results.areaUnderROC, results.weightedPrecision, results.weightedRecall, results.weightedFMeasure()

(1.0, 1.0, 1.0, 1.0, 1.0)